In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6ea28eb138586fddabafa60fdda7f48501d61c999f61424fe5096da047d9cffe
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [56]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
mydata = spark.read.format("csv").option("header","true").load("/content/drive/MyDrive/pyspark_crachcourse/original.csv")

In [59]:
mydata.show(6)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
+---+----------+----------+------+---------------+-----

In [60]:
from pyspark.sql.functions import *

In [61]:
mydata2 = mydata.withColumn(
    "Clean_City",
    when(col("City").isNull(), "unknown").otherwise(col("City"))
)

In [62]:
mydata2.show(5)

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|   Clean_City|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Bulgan|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|      unknown|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|    Mytishchi|
+---+----------+----------+------+-------------+----------------

In [63]:
mydata3 = mydata2.filter(
    col("JobTitle").isNotNull()
    )

In [64]:
mydata3.show(5)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     Clean_City|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
+---+----------+----------+-----

In [65]:
mydata3.select(first("Salary")).first()[0]

'$57438.18'

In [66]:
mydata3.select(expr("substring(first(Salary), 2, 100)")).first()[0]

'57438.18'

In [67]:
mydata3 = mydata3.withColumn(
    "Clean_Salary",
    expr("cast(substring(Salary,2,100) as float)")
    )

In [68]:
mydata3.show(6)

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     Clean_City|Clean_Salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [69]:
 mean = mydata3.groupBy().agg(avg(col("Clean_Salary"))).collect()[0][0]

In [70]:
mean

55516.32088199837

In [71]:
mydata3 = mydata3.withColumn(
    "New_Salary",
    when(col("Clean_Salary").isNull(),mean).otherwise(col("Clean_Salary"))
)

In [72]:
mydata3.show(3)

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+------------+--------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|   Clean_City|Clean_Salary|    New_Salary|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+------------+--------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    Nowa Ruda|    57438.18| 57438.1796875|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Bulgan|     62846.6| 62846.6015625|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Divnomorskoye|    61489.23|61489.23046875|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+------------+-

In [73]:
import pyspark.sql.functions as sqlfunc

genders = mydata3.groupBy('gender').agg(sqlfunc.avg("New_Salary").alias("AvgSalary"))
genders.show()

+------+------------------+
|gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



In [74]:
genders_ = mydata3.groupBy('Gender').agg(avg(col('New_Salary')).alias('AvgSalary'))
genders_.show()

+------+------------------+
|Gender|         AvgSalary|
+------+------------------+
|Female|55677.250125558036|
|  Male| 55361.09385573019|
+------+------------------+



# ***Get The Average Salary Per Each Job_title For Each Gernder***

In [76]:
# Get The Average Salary Per Each Job_title For Each Gernder

mydata4 = mydata3.withColumn(
    "Female_Salary",
    when(mydata3.gender == 'Female',mydata3.New_Salary).otherwise(lit(0))
    )

mydata5 = mydata4.withColumn(
    "Male_Salary",
    when(mydata3.gender == 'Male',mydata3.New_Salary).otherwise(lit(0))
    )

In [77]:
mydata5.show(3)

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+------------+--------------+-------------+--------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|   Clean_City|Clean_Salary|    New_Salary|Female_Salary|   Male_Salary|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+-------------+------------+--------------+-------------+--------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    Nowa Ruda|    57438.18| 57438.1796875|57438.1796875|           0.0|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|       Bulgan|     62846.6| 62846.6015625|62846.6015625|           0.0|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|Divnomorskoye|    61489.23|6

In [78]:
df = mydata5.groupBy("JobTitle").agg(
    sqlfunc.avg("Female_Salary").alias("Final_Female_Salary"),
    sqlfunc.avg("Male_Salary").alias("Final_Male_Salary")
)

In [80]:
df.show(5)

+--------------------+-------------------+------------------+
|            JobTitle|Final_Female_Salary| Final_Male_Salary|
+--------------------+-------------------+------------------+
|Systems Administr...|    50590.474609375|  15540.9501953125|
|   Media Manager III| 29586.436197916668|17381.920572916668|
|  Recruiting Manager| 34848.452473958336|  26383.4951171875|
|       Geologist III|       31749.046875|    12830.75390625|
|        Geologist II|                0.0|   43293.865234375|
+--------------------+-------------------+------------------+
only showing top 5 rows



In [82]:
df.withColumn(
    "delta",
    df.Final_Female_Salary - df.Final_Male_Salary
).show()

+--------------------+-------------------+------------------+-------------------+
|            JobTitle|Final_Female_Salary| Final_Male_Salary|              delta|
+--------------------+-------------------+------------------+-------------------+
|Systems Administr...|    50590.474609375|  15540.9501953125|   35049.5244140625|
|   Media Manager III| 29586.436197916668|17381.920572916668|       12204.515625|
|  Recruiting Manager| 34848.452473958336|  26383.4951171875|  8464.957356770836|
|       Geologist III|       31749.046875|    12830.75390625|     18918.29296875|
|        Geologist II|                0.0|   43293.865234375|   -43293.865234375|
|Database Administ...|                0.0|     52018.4609375|     -52018.4609375|
|   Financial Analyst|    23353.776953125|       39606.05625|   -16252.279296875|
|  Analyst Programmer|   16406.1287109375|  21042.9634765625| -4636.834765625001|
|Software Engineer II|                0.0|      74782.640625|      -74782.640625|
|       Accounta

# ***Get The Average Salary Per City***

In [86]:
mydata5.groupBy("City").agg(
    sqlfunc.avg("New_Salary").alias("avgsalary")
).sort(
    col("avgsalary").desc()
    ).show()

+-----------------+-------------+
|             City|    avgsalary|
+-----------------+-------------+
|        Mesopotam|  99948.28125|
|       Zhongcheng| 99942.921875|
|           Caxias|99786.3984375|
|      Karangtawar|99638.9921875|
|        Itabaiana|  99502.15625|
|           Pasian|  99421.34375|
|           Webuye| 99368.546875|
|      Yuktae-dong| 99250.828125|
|           Zinder|  99222.84375|
|   Timiryazevskiy|   99142.9375|
|        Sawahbaru|99013.7109375|
|          Madimba|98737.8671875|
|         Huangshi|  98690.34375|
|          Gharyan|   98679.3125|
|         Yŏnan-ŭp| 98628.609375|
|     Wringinputih|98603.8203125|
|Monte da Boavista|  98586.71875|
|          Klukeng|98439.4921875|
|         Murmashi|  98226.15625|
|        Fox Creek|      98138.0|
+-----------------+-------------+
only showing top 20 rows

